#Generació d'una cadena de caràcters fent servir un Algorisme Genètic

En aquesta lliçó prendrem com a objectiu la generació d'una cadena de text  mitjançant l'ús d'un algorisme genètic per evolucionar la població fins a arribar a la cadena de text òptima. La qualitat de les cadenes generades es mesurarà mitjançant una funció d'aptitud que comprovi com s'apropen les cadenes de la població a la cadena desitjada.



In [ ]:
import random

Iniciarem el codi definint quina és la cadena òptima que l'algorisme ha d'aconseguir generar. També definirem els paràmetres essencials com la mida de la població o el nombre màxim de generacions que durà a terme l'algorisme.

In [ ]:
MIDA_POBLACIO = 11

# Cadena que volem que es generi
OBJECTIU = "10001111"

# Gens que es poden incorporar dins del cromosoma.
#En aquest cas, l'abecedari en majúscules i minúscules i l'espai.
GENS = "01"

# Màxim nombre de generacions fins que parem l'algorisme en cas de que no es trobi
#la solució perfecta.
MAXGENERACIONS = 1000


Seguidament, definirem una funció per crear un cromosoma. En aquest cas, un cromosoma és una cadena de caràcters (cada caràcter és un gen) de la mateixa longitud que la cadena òptima.

In [ ]:
def crear_cromosoma():
  nou_cromosoma = []
  for i in range(0,len(OBJECTIU)):
    nou_cromosoma.append(random.choice(GENS))
  return nou_cromosoma

També definirem una funció per a mutar cromosomes, amb la finalitat d'aportar més riquesa i variabilitat a la població. La mutació serà la substitució d'un gen (caràcter) del cromosoma (la cadena) per un altre.

In [ ]:
def mutar(cromosoma, taxa_mutacio):
    cromosoma_mutat = []
    for gen in cromosoma:
        if random.uniform(0, 1) < taxa_mutacio:
            # Si la probabilitat de mutació és satisfeta, canviem el gen
            cromosoma_mutat.append(random.choice(GENS))
        else:
           # Altrament, retornem el cromosoma tal com estava
            cromosoma_mutat.append(gen)
    return cromosoma_mutat

Per a aparellar cadenes, farem servir l'estratègia de *Single-Point Crossover*. Per tant, seleccionarem de forma aleatòria un punt de tall, i conformarem el nou cromosoma agafant des del tall i cap a la dreta d'un dels pares i des del tall i cap a l'esquerra de l'altre pare. Serà després de generar el nou cromosoma que cridarem a la funció que hem definit anteriorment.

In [ ]:
def aparellar(par1, par2):
  '''
  Apareació per Single-Point Crossover
  '''
  # Punt de tall per la recombinació
  punt_tall = 4 # random.randint(0, min(len(par1), len(par2)))

  # Obtenir grups de cromosomes per a cada rang per generar cada fill
  high_par1 = par1[:punt_tall]
  low_par1 = par1[punt_tall:]
  high_par2 = par2[:punt_tall]
  low_par2 = par2[punt_tall:]
  high_par1.extend(low_par2)
  high_par2.extend(low_par1)

  # Aplicar mutació al cromosoma resultant. Li passarem una taxa de probabilitat de 5%



  return mutar(high_par1, 0.05),mutar(high_par2, 0.05)
  # fill1,fill2 = aparellar(['0','1','0','1','1','1','0','0'],['1','1','0','1','0','1','1','0'])
  # fill1 = "".join(fill1)
  # fill2 = "".join(fill2)
  # assert fill1 == '01010110'
  # assert fill2 == '01100101'


L'aptitud es mesurarà de forma que es contin els caràcters diferents entre el cromosoma que s'està avaluant i la cadena *objectiu*. Per tant, una solució perfecta tindrà una aptitud de 0.

In [ ]:
def calcular_aptitud(cromosoma, objectiu):
  aptitud = 0
  objectiu_int=int(objectiu, 2)
  high=int("".join(cromosoma[:-4]),2)
  low=int("".join(cromosoma[4:]),2)

  return abs(objectiu_int-(high*low))

Definides totes les funcions anteriors, ja es pot començar l'algorisme.

In [ ]:
generacio = 1
trobat = False
poblacio = []

print("Població inicial:")
for _ in range(MIDA_POBLACIO):
  cromosoma = crear_cromosoma()
  poblacio.append((cromosoma, calcular_aptitud(cromosoma, OBJECTIU)))
  # Descomentar la següent línia de codi si es vol analitzar la població inicial
  # print((cromosoma, calcular_aptitud(cromosoma, OBJECTIU)))

print("Iniciem el procés d'aparellament:")

while not trobat:
  # La població (que és una llista d'individus) s'ordena segons el segon
  #element de cada tupla, que correspon a la seva aptitud.
  # Això significa que els individus amb menor aptitud estan a la part frontal de la llista.
  poblacio = sorted(poblacio, key=lambda x: x[1])

  #Si el primer element de la població té una aptitud de 0, significa que ja hem
  #trobat la cadena objectiu, i per tant podem parar l'algorisme.
  if poblacio[0][1] == 0:
      trobat = True
      break

  # Es crea una nova generació afegint una part de la població actual
  #(els millors individus) a la nova generació.
  # Aquesta part (el 10% de la població) se selecciona basant-se en l'aptitud,
  #ja que la població ja està ordenada.
  nova_generacio = []
  poblacio_mantinguda = int((0.1 * MIDA_POBLACIO))
  nova_generacio.extend(poblacio[:poblacio_mantinguda])

  # El 90% de població restant se substitueix. Per a fer-ho, es fan parelles per a criar noves cadenes.
  for _ in range(0, int((0.9 * MIDA_POBLACIO))):
      #S'agafarà de manera aleatòria una de les 10 mostres amb un valor més petit d'aptitud.
      pare1 = random.choice(poblacio[:10])
      pare2 = random.choice(poblacio[:10])
      fill1,fill2 = aparellar(pare1[0], pare2[0])
      nova_generacio.append((fill1, calcular_aptitud(fill1, OBJECTIU)))
      nova_generacio.append((fill2, calcular_aptitud(fill2, OBJECTIU)))

  poblacio = nova_generacio
  # Es busca la millor solució (la que té l'aptitud més petita)
  millor_solucio = min(poblacio, key=lambda ind: ind[1])
  print("Nombre de generació: {} Millor cadena de la generació: {} Aptitud: {}".format(generacio, "".join(millor_solucio[0]), millor_solucio[1]))

  generacio += 1
  if (generacio > MAXGENERACIONS):
    millor = "".join((millor_solucio[0]))
    print(f"No s'ha trobat cap generació")
    break





Població inicial:
Iniciem el procés d'aparellament:
Nombre de generació: 1 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 2 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 3 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 4 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 5 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 6 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 7 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 8 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 9 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 10 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 11 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 12 Millor cadena de la generació: 11101010 Aptitud: 3
Nombre de generació: 13 Millor cadena de la generació: 11